In [1]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
#https://en.wikipedia.org/wiki/Shapiro–Wilk_test

In [3]:
def count_total(df_):
    df = df_.copy()
    common_cols = df.columns[:4]
    df['total'] = sum([df[c] for c in df.columns if c not in common_cols])
    df.drop(columns=[c for c in df.columns if c not in (list(common_cols)+['total'])], inplace=True)
    return df


In [4]:
# file = 'DAY_AHEAD'
file = 'REAL_TIME'
pv = pd.read_csv(f'/app/saves/timeseries_data_files/PV/{file}_pv.csv')
rtpv = pd.read_csv(f'/app/saves/timeseries_data_files/RTPV/{file}_rtpv.csv')
wind = pd.read_csv(f'/app/saves/timeseries_data_files/WIND/{file}_wind.csv')
load = pd.read_csv(f'/app/saves/timeseries_data_files/Load/{file}_regional_Load.csv')
hydro = pd.read_csv(f'/app/saves/timeseries_data_files/Hydro/{file}_hydro.csv')


In [5]:
wind_total = count_total(wind)
load_total = count_total(load)
pv_total = count_total(pv)
rtpv_total = count_total(rtpv)
hydro_total = count_total(hydro)

In [6]:
total = load_total.copy()
total['total'] -= wind_total['total']
total['total'] -= pv_total['total']
total['total'] -= rtpv_total['total']
total['total'] -= hydro_total['total']
total=total.groupby(['Year', 'Month', 'Day']).sum().reset_index().drop(columns=['Period'])
# total = total[total['Month'] == 3]

In [7]:
wind

,Year,Month,Day,Period,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1
0,2020,1,1,1,146.0,782.7,826.2,698.4
1,2020,1,1,2,143.9,787.8,822.8,701.6
2,2020,1,1,3,144.2,785.8,827.1,701.7
3,2020,1,1,4,143.4,783.4,834.3,702.1
4,2020,1,1,5,143.7,782.9,832.3,701.3
...,...,...,...,...,...,...,...,...
105403,2020,12,31,284,1.2,19.8,121.8,126.2
105404,2020,12,31,285,0.6,20.1,120.2,126.1
105405,2020,12,31,286,1.0,18.6,114.0,128.9
105406,2020,12,31,287,0.8,16.9,122.4,137.8


In [8]:
stat, p = stats.shapiro(total['total'].values)
alpha = 0.05  # significance level
print('statistic = ', p)
if p > alpha:
    print("Sample looks Gaussian (fail to reject H0)")
else:
    print("Sample does not look Gaussian (reject H0)")
SW_results = pd.DataFrame(columns=['Month', 'p-value', "SW test decision (0.05 significance level)"])
for i, m in enumerate(range(1, 13)):
    stat, p = stats.shapiro(total[total['Month'] == m]['total'].values)
    # stat, p = stats.shapiro(total['total'].values)
    SW_results.loc[i] = [m, p, "Sample looks Gaussian (fail to reject H0)"] if p > alpha else [m, p, "Sample does not look Gaussian (reject H0)"]

statistic =  0.0003442287904920071
Sample does not look Gaussian (reject H0)


In [9]:
SW_results

,Month,p-value,SW test decision (0.05 significance level)
0,1,0.108437,Sample looks Gaussian (fail to reject H0)
1,2,0.167564,Sample looks Gaussian (fail to reject H0)
2,3,0.429054,Sample looks Gaussian (fail to reject H0)
3,4,0.178237,Sample looks Gaussian (fail to reject H0)
4,5,0.140386,Sample looks Gaussian (fail to reject H0)
5,6,0.162291,Sample looks Gaussian (fail to reject H0)
6,7,0.334059,Sample looks Gaussian (fail to reject H0)
7,8,0.000005,Sample does not look Gaussian (reject H0)
8,9,0.056670,Sample looks Gaussian (fail to reject H0)
9,10,0.562937,Sample looks Gaussian (fail to reject H0)
